<a href="https://colab.research.google.com/github/namtoptall/DataScience/blob/main/%5BVN%5D_Text_Classification_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Trong notebook này, chúng ta sẽ thử quy trình lập trình RNN với Keras để phân loại các câu văn bản.

I.   **Trước tiên**, chúng ta sẽ nhập các thư viện hữu ích.

II.   **Sau đó**, chúng ta sẽ load dữ liệu và tạo ma trận word embedding bằng Glove.

III.  **Chúng ta sẽ thử một mô hình RNN đơn giản** rồi đánh giá chất lượng của nó.

IV. Cuối cùng, chúng ta sẽ sử dụng các kỹ thuật để gia tăng độ chính xác của mô hình.

**Task 1:** Thiết lập Fre GPU trong notebook này.

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Gắn Google Drive cục bộ
**Task 2:** Gắn Google vào Google Colab Driver.


In [2]:
## VIẾT CODE cho task 2 ở đây:
#mount drive content
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# I. Nhập tất cả các thư viện hữu ích.

In [3]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import tensorflow.keras
import datetime
from tensorflow.keras import backend as K
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
# import tensorflow_addons as tfa

from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.metrics import confusion_matrix as CM
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import matplotlib.pyplot as plot
import seaborn as sn

**Task 3**: Copy tập dữ liệu từ Google Drive vào Colab

In [4]:
# copy txt file
!cp /content/drive/MyDrive/asm2/glove.6B.50d.txt.zip .

In [5]:
# unzip files
!unzip glove.6B.50d.txt.zip -d .

Archive:  glove.6B.50d.txt.zip
  inflating: ./glove.6B.50d.txt      


In [6]:
# open the first 3 lines of the dataset
text = open('glove.6B.50d.txt', 'r')
for i in range(10):
    print(text.readline())

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581

, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392

. 0.15164 0.30177 -0.16763 0.17684 0.31719 0.33973 -0.43478 -0.31086 -0.44999 -0.29486 0.16608 0.11963 -0.41328 -0.423

In [7]:
# check the train
!cp /content/drive/MyDrive/asm2/train.csv.zip .

In [8]:
# unzip the train
!unzip /content/drive/MyDrive/asm2/train.csv.zip

Archive:  /content/drive/MyDrive/asm2/train.csv.zip
  inflating: train.csv               


In [9]:
data = pd.read_csv('train.csv')
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [10]:
print(data.columns)

Index(['qid', 'question_text', 'target'], dtype='object')


# II. Load dữ liệu.

## Về tập dữ liệu.
Câu hỏi không hợp lệ là những câu hỏi nhằm đưa ra một tuyên bố thay vì tìm kiếm những câu trả lời hữu ích. Một số đặc điểm cho thấy câu hỏi không hợp lệ gồm:

* Có giọng điệu không trung lập.
* Có tính chê bai hoặc kích động.
* Không có căn cứ thực tế.
* Sử dụng nội dung khiêu dâm (loạn luân, thú tính, ấu dâm) để gây sốc, không phải để tìm kiếm câu trả lời xác thực.

Dữ liệu bao gồm câu hỏi đã đề ra và liệu nó có được xác định là không hợp lệ hay không (target = 1).

**Task 4**: Load tập dữ liệu.
* Load dữ liệu từ file CSV.
* Xóa tất cả các hàng có giá trị NA.
* Chia dữ liệu thành 3 tập: Tập huấn luyện, tập kiểm định và tập kiểm tra (0.9/0.05/0.05, random_seed = 9) với cùng một tỷ lệ số dữ liệu giữa mỗi lớp.
* In ra mô tả của tập dữ liệu này.




In [11]:
from sklearn.model_selection import train_test_split
def load_data(data_link):
    '''
    input: data link.
    output:
        train_set, validation_set và test_set(0.95/0.05/0.05) mà không có các giá trị NA.
    '''
    ## VIẾT CODE cho task 4 ở đây:
    data = pd.read_csv(data_link)
    data = data.dropna()
    # Chia dữ liệu thành tập huấn luyện và tập tạm thời (0.9/0.1)
    train, temp = train_test_split(data, test_size=0.1, stratify=data['target'], random_state=9)
    # Chia tập tạm thời thành tập kiểm định và tập kiểm tra (0.05/0.05)
    validation, test = train_test_split(temp, test_size=0.5, stratify=temp['target'], random_state=9)
    return train, validation, test

train_set, validation_set, test_set = load_data('train.csv')
print(train_set['target'].describe())
print(validation_set['target'].describe())
print(test_set['target'].describe())

count    1.175509e+06
mean     6.187022e-02
std      2.409198e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: target, dtype: float64
count    65306.000000
mean         0.061863
std          0.240908
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: target, dtype: float64
count    65307.000000
mean         0.061877
std          0.240934
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: target, dtype: float64


# Mã hóa dữ liệu văn bản.
Hãy khai báo một số tham số cơ bản trước:

In [12]:
embed_size = 50 # mỗi vectơ từ lớn bao nhiêu
max_features = 20000 # cần sử dụng bao nhiêu từ duy nhất (tức là số hàng trong vectơ embedding)
max_len = 50 # sử dụng số từ tối đa trong câu hỏi


**Task 5:** Mã hóa tập dữ liệu bằng Tokenizer và vectơ biểu diễn one-hot.
* Mã hóa văn bản (cột question_text) bằng cách chuyển từng question text thành danh sách chỉ mục từ bằng [Tokenizer](https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do) với **max_features** và tất cả các câu văn bản từ tập huấn luyện và tập kiểm định.
* Chuyển từng danh sách chỉ mục từ thành độ dài như nhau - **max_len** (có cắt tỉa hoặc đệm nếu cần) bằng cách sử dụng [pad_sequences](https://keras.io/preprocessing/sequence/).
* Mã hóa nhãn (cột nhãn) bằng cách sử dụng hàm [to_categorical](https://keras.io/utils/) trong Keras.

In [13]:
def encoding_textdata(train_set, validation_set, test_set, max_features, max_len):
    '''
    Input:
    - Train/validation/test dataset.
    - max_features, max_len.
    Output:
    - X train/validation/test, y train/validation/test.
    - Tokenizer.
    '''
    ## VIẾT CODE cho task 5 ở đây:
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(train_set['question_text'])

    X_tr = tokenizer.texts_to_sequences(train_set['question_text'])
    X_va = tokenizer.texts_to_sequences(validation_set['question_text'])
    X_te = tokenizer.texts_to_sequences(test_set['question_text'])

    X_tr = pad_sequences(X_tr, maxlen=max_len)
    X_va = pad_sequences(X_va, maxlen=max_len)
    X_te = pad_sequences(X_te, maxlen=max_len)

    y_tr = train_set['target']
    y_va = validation_set['target']
    y_te = test_set['target']

    return (X_tr, y_tr), (X_va, y_va), (X_te, y_te), tokenizer # Return values from the outer function

(X_tr, y_tr), (X_va, y_va), (X_te, y_te), tokenizer = encoding_textdata(train_set, validation_set, test_set, max_features, max_len) # Call the outer function


**Task 6**: Tạo ma trận word embedding.
* Đầu tiên, hãy viết một hàm để load [GloVe dictionary.](https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db)
* Sau đó, tạo ma trận word embedding bằng GloVe dictionary với các tham số sau:
    - Shape của ma trận word embedding: (Number of word, embed_size).
    - Embed size: 50.
    - Number of words: Tối thiểu của (max_features, len(word_index)), trong khi word_index là dictionary của từ chứa trong tokenizer.
    - Nếu một từ xuất hiện trong GloVe dictionary, chúng ta nên lấy giá trị khởi tạo của nó như trong GloVe dictionary. Nếu không, hãy lấy một giá trị ngẫu nhiên bình thường với mean và std làm mean và std của giá trị GloVe dictionary.
    



In [14]:
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def get_GloVe_dict(GloVe_link):
    '''
    input: GloVe link.
    output: GloVe dictionary.
    '''
    GloVe_dict = {}
    with open(GloVe_link, encoding="utf8") as f:
        for line in f:
            word, coefs = get_coefs(*line.strip().split())
            GloVe_dict[word] = coefs
    return GloVe_dict
GloVe_dict = get_GloVe_dict('glove.6B.50d.txt')

In [15]:
for key, value in GloVe_dict.items():
    print(key, value)
    break

the [ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]


In [16]:
def create_embedding_matrix(GloVe_dict, tokenizer, max_features, embed_size):
    '''
    input: GloVe dictionary, tokenizer từ tập huấn luyện và tập kiểm định, số lượng đặc trưng tối đa, kích thước embedding.
    output: Word embedding matrix.
    '''
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    embedding_matrix = np.zeros((num_words, embed_size))

    all_embs = np.stack(list(GloVe_dict.values()))  # Chuyển đổi thành danh sách
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    emb_size = all_embs.shape[1]

    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = GloVe_dict.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            embedding_matrix[i] = np.random.normal(emb_mean, emb_std, emb_size)

    return embedding_matrix
embed_size = 50
max_features = 20000

embedding_matrix = create_embedding_matrix(GloVe_dict, tokenizer, max_features, embed_size)

In [17]:
print(embedding_matrix.shape)

(20000, 50)


# III. Lập mô hình
Chúng ta cần hoàn thành một số bước:

Xây dựng mô hình.

Biên dịch mô hình.

Huấn luyện/khớp dữ liệu với mô hình.

Đánh giá mô hình trên tập kiểm tra.

## Xây dựng mô hình
**Task 7:** Chúng ta có thể xây dựng một mô hình dễ dàng gồm các layer khác nhau như:
* Layer [Embedding](https://keras.io/layers/embeddings/) với max_features, embed_size và embedding_matrix.
* [Bidirectional LSTM layer](https://keras.io/examples/nlp/bidirectional_lstm_imdb/?fbclid=IwAR3fEd6aWyeIDEhZSspjtCRiP0c0Jnz5-XdnUHQYwX8Tp8k9Ni4I8Q5tP9o) với số lượng trạng thái ẩn = 50, dropout_rate = 0.1 và recurrent_dropout_rate = 0.1.
* GlobalMaxPool1D.
* Dense với số nút = 50, activation = 'relu'.
* Dropout với rate = 0.1.
* Final dense với số nút = số class, activation = 'sigmoid'.

In [18]:
def create_model(max_len, max_features, embed_size):
    '''
    input: max_len, max_features, embed_size
    output: model.
    '''
    ## VIẾT CODE cho task 7 ở đây:
    inputs = Input(shape=(max_len,))
    embedding = Embedding(max_features,
                            embed_size,
                            weights=[embedding_matrix],trainable=False)(inputs)
    lstm = Bidirectional(LSTM(50,
                              return_sequences=True,
                              dropout=0.1,
                              recurrent_dropout=0.1))(embedding)
    pooling = GlobalMaxPool1D()(lstm)
    dense = Dense(50,activation='relu')(pooling)
    dropout = Dropout(0.1)(dense)
    output = Dense(1,activation='sigmoid')(dropout)
    model = Model(inputs=inputs, outputs=output)

    return model

model = create_model(max_len, max_features, embed_size)

**Task 8:** Biên dịch mô hình và thiết lập callback. Sau đó in ra model summary.
* [Biên dịch](https://keras.io/models/model/#compile) mô hình với Adam Optimizer, lr = 1e-2, loss phù hợp cho bài toán phân loại nhị phân và ["F1-score"](https://github.com/tensorflow/addons/issues/825) là phép đo.
* In ra model summary.

In [19]:
from tensorflow.keras.optimizers import Adam

def optimize(model):
    model.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = optimize(model)
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 50, 50)              │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 50, 100)             │          40,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 100)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,045,501 (3.99 MB)

 Trainable params: 45,501 (177.74 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

None


**Task 9**: Thiết lập callback.
* Tạo [tensorboard callback](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks) để lưu logs.
* Tạo [checkpoint callback](https://machinelearningmastery.com/check-point-deep-learning-models-keras/) để lưu checkpoint với độ chính xác tốt nhất sau mỗi epoch.
* Tạo [ReduceLROnPlateau](https://keras.io/callbacks/#reducelronplateau) callback với factor=0.3, patience=1 và "Validation F1-score" monitor.
* Tạo [early stopping callback](https://keras.io/callbacks/#earlystopping) với patience=7, mode = 'max' và "Validation F1-score" monitor.



In [20]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Tạo callback cho TensorBoard
logs_name = 'training_logs'
tensorboard_callback = TensorBoard(log_dir=logs_name, histogram_freq=1)

In [21]:
# Hàm callback_model
def callback_model(checkpoint_name, logs_name):
    tensorboard_callback = TensorBoard(log_dir=logs_name, histogram_freq=1)
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_name + '.keras',
                                          save_best_only=True,
                                          monitor='val_loss',
                                          mode='min',
                                          verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1, mode='min')
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=7, mode='min', verbose=1)

    return [tensorboard_callback, checkpoint_callback, reduce_lr_callback, early_stopping_callback]

checkpoint_name = 'weights.best'
callbacks_list = callback_model(checkpoint_name, logs_name)


In [22]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 50, 50)              │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 50, 100)             │          40,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 100)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,045,501 (3.99 MB)

 Trainable params: 45,501 (177.74 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

**Task 10:** Huấn luyện mô hình.

* Huấn luyện mô hình với 20 epoch với batch_size = 4096.
* Trả về mô hình có trọng số checkpoint tốt nhất.

*Gợi ý*: Trước tiên hãy khớp mô hình, sau đó reload mô hình (hàm load_model) với trọng số checkpoint tốt nhất.

In [ ]:
def train_model(model, callbacks_list):
    '''
    Input:
        Mô hình và callback list,
    Return:
        Mô hình với trọng số checkpoint tốt nhất.
    '''
    ## VIẾT CODE cho task 10 ở đây:
    model.fit(X_tr, y_tr,
              batch_size=4096,
              epochs=20,
              validation_data=(X_va, y_va),
              callbacks=callbacks_list)
    model.load_weights('weights.best.keras')
    return model

model = train_model(model, callbacks_list)

Epoch 1/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9210 - loss: 0.2072
Epoch 1: val_loss improved from inf to 0.12551, saving model to weights.best.keras
287/287 ━━━━━━━━━━━━━━━━━━━━ 650s 2s/step - accuracy: 0.9211 - loss: 0.2070 - val_accuracy: 0.9515 - val_loss: 0.1255 - learning_rate: 0.0100
Epoch 2/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9503 - loss: 0.1283
Epoch 2: val_loss improved from 0.12551 to 0.11661, saving model to weights.best.keras
287/287 ━━━━━━━━━━━━━━━━━━━━ 648s 2s/step - accuracy: 0.9503 - loss: 0.1283 - val_accuracy: 0.9547 - val_loss: 0.1166 - learning_rate: 0.0100
Epoch 3/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9534 - loss: 0.1197
Epoch 3: val_loss improved from 0.11661 to 0.11360, saving model to weights.best.keras
287/287 ━━━━━━━━━━━━━━━━━━━━ 611s 2s/step - accuracy: 0.9534 - loss: 0.1197 - val_accuracy: 0.9558 - val_loss: 0.1136 - learning_rate: 0.0100
Epoch 4/20
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy:

**Task 11:** Hiển thị tensorboard trong notebook.

In [ ]:
## VIẾT CODE cho task 11 ở đây:
%load_ext tensorboard
%tensorboard --logdir logs_name

In [ ]:
# plot model
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

**Task 12:** Dự đoán trên tập kiểm tra.

* Hoàn thành hàm get_prediction_classes.
* In ra precision, recall và F1 score.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

def get_prediction_classes(model, X, y):
    '''
    Input:
        Mô hình và tập dữ liệu dự đoán.
    Return:
        Prediction list và groundtruth list với predicted classes.
    '''
    y_pred = model.predict(X)
    predictions = (y_pred > 0.5).astype(int).flatten()
    groundtruths = y.to_numpy().flatten()
    return predictions, groundtruths

# Dự đoán trên tập kiểm tra
test_predictions, test_groundtruths = get_prediction_classes(model, X_te, y_te)

# In ra precision, recall và F1 score
print(f"Precision: {precision_score(test_groundtruths, test_predictions)}")
print(f"Recall: {recall_score(test_groundtruths, test_predictions)}")
print(f"F1 Score: {f1_score(test_groundtruths, test_predictions)}")


**Task 13:** Thực hiện kết quả dự đoán trên tập kiểm tra bằng cách sử dụng ma trận nhầm lẫn. Hãy nhớ hiển thị tên lớp trong ma trận nhầm lẫn.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def plot_confusion_matrix(predictions, groundtruth, class_names):
    ## VIẾT CODE cho task 13 ở đây:
    cm = confusion_matrix(groundtruth, predictions)
    df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
    plt.figure(figsize=(10,7))
    sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plot.show()
class_names = ['valid', 'invalid']
plot_confusion_matrix(test_predictions, test_groundtruths, class_names)

**Task 14**: Tinh chỉnh mô hình - tinh chỉnh mô hình bằng cách sử dụng một số phương pháp sau:
* Tăng tối đa epoch, thay đổi batch size.
* Thay thế LSTM bằng các nút GRU và kiểm tra xem nó có thay đổi gì không.
* Thêm một layer LSTM/GRU khác hoặc thay thế nó bằng mô-đun Attention/Transformers, xem có cải thiện gì không.
* Thử với Dense layer (add/# units/...).
* Tìm các quy tắc tiền xử lý mà bạn có thể thêm để cải thiện chất lượng dữ liệu.
* Find another GloVe dictionary. Tìm một GloVe dictionary khác.
Yêu cầu: F1 score phải tăng thêm 2-3%.

### Tăng số epochs

In [ ]:
## VIẾT CODE cho task 14 ở đây:
# tăng số epoch
def train_model(model, callbacks_list):
    model.fit(X_tr, y_tr,
              batch_size=4096,
              epochs=25,
              validation_data=(X_va, y_va),
              callbacks=callbacks_list)
    model.load_weights('weights.best.keras')
    return model

model_25_epoch = train_model(model, callbacks_list)

### Thay thế LSTM bằng GRU


In [ ]:
model_25_epoch.evaluate(X_te, y_te)

In [ ]:
from tensorflow.keras.layers import GRU
def model_with_gru(max_len, max_features, embed_size, embedding_matrix):
    '''
    input: max_len, max_features, embed_size, embedding_matrix
    output: model.
    '''
    max_len = 50
    max_features = 20000
    embed_size = 50
    inputs = Input(shape=(max_len,))
    embedding = Embedding(max_features,
                            embed_size,
                            weights=[embedding_matrix],trainable=False)(inputs)
    gru = Bidirectional(GRU(50,
                              return_sequences=True,
                              dropout=0.1,
                              recurrent_dropout=0.1))(embedding)
    pooling = GlobalMaxPool1D()(gru)
    dense = Dense(50,activation='relu')(pooling)
    dropout = Dropout(0.1)(dense)
    output = Dense(1,activation='sigmoid')(dropout)

    model = Model(inputs=inputs, outputs=output)

    return model

modelgru = model_with_gru(max_len, max_features, embed_size, embedding_matrix)
#compile model
modelgru.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
#train model
modelgru.fit(X_tr, y_tr,
              batch_size=4096,
              epochs=25,
              validation_data=(X_va, y_va),
              callbacks=callbacks_list)

In [ ]:
# evaluate the model
test_predictions_gru, test_groundtruths_gru = get_prediction_classes(modelgru, X_te, y_te)
print(f"Precision: {precision_score(test_groundtruths_gru, test_predictions_gru)}")
print(f"Recall: {recall_score(test_groundtruths_gru, test_predictions_gru)}")
print(f"F1 Score: {f1_score(test_groundtruths_gru, test_predictions_gru)}")

In [ ]:
# print
plot_confusion_matrix(test_predictions_gru, test_groundtruths_gru, class_names)

### Thêm một layer LSTM/GRU khác


In [ ]:
def model_with_more(max_len, max_features, embed_size, embedding_matrix):
    max_len = 50
    max_features = 20000
    embed_size = 50
    inputs = Input(shape=(max_len,))
    embedding = Embedding(max_features,
                            embed_size,
                            weights=[embedding_matrix],trainable=False)(inputs)
    gru = Bidirectional(GRU(50,
                          return_sequences=True,
                          dropout=0.2,
                          recurrent_dropout=0.1))(embedding)
    gru2 = Bidirectional(GRU(50,
                          return_sequences=True,
                          dropout=0.1,
                          recurrent_dropout=0.1))(gru)
    pooling = GlobalMaxPool1D()(gru2)
    dense = Dense(50,activation='relu')(pooling)
    dropout = Dropout(0.1)(dense)
    output = Dense(1,activation='sigmoid')(dropout)

    model = Model(inputs=inputs, outputs=output)

    return model

model_with_more = model_with_more(max_len, max_features, embed_size, embedding_matrix)
model_with_more.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
model_with_more.summary()

In [ ]:
model_with_more.fit(X_tr, y_tr,
              batch_size=4096,
              epochs=25,
              validation_data=(X_va, y_va),
              callbacks=callbacks_list)

In [ ]:
test_predictions_more, test_groundtruths_more = get_prediction_classes(model_with_more, X_te, y_te)
print(f"Precision: {precision_score(test_groundtruths_more, test_predictions_more)}")
print(f"Recall: {recall_score(test_groundtruths_more, test_predictions_more)}")
print(f"F1 Score: {f1_score(test_groundtruths_more, test_predictions_more)}")

In [ ]:
plot_confusion_matrix(test_predictions_more, test_groundtruths_more, class_names)

### Thử với các lớp Dense khác nhau

In [ ]:
def model_with_denses(max_len, max_features, embed_size, embedding_matrix):
    max_len = 50
    max_features = 20000
    embed_size = 50
    inputs = Input(shape=(max_len,))
    embedding = Embedding(max_features,
                            embed_size,
                            weights=[embedding_matrix],trainable=False)(inputs)
    lstm = Bidirectional(LSTM(50,
                              return_sequences=True,
                              dropout=0.1,
                              recurrent_dropout=0.1))(embedding)
    pooling = GlobalMaxPool1D()(lstm)
    dense1 = Dense(100,activation='relu')(pooling)
    dropout1 = Dropout(0.1)(dense1)
    dense2 = Dense(50,activation='relu')(dropout1)
    dropout2 = Dropout(0.1)(dense2)
    dense3 = Dense(25,activation='relu')(dropout2)
    dropout3 = Dropout(0.1)(dense3)
    output = Dense(1,activation='sigmoid')(dropout3)

    model = Model(inputs=inputs, outputs=output)

    return model

denses_model = model_with_denses(max_len, max_features, embed_size, embedding_matrix)
denses_model.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
denses_model.summary()


In [ ]:
denses_model.fit(X_tr, y_tr,
              batch_size=4096,
              epochs=25,
              validation_data=(X_va, y_va),
              callbacks=callbacks_list)

In [ ]:
test_predictions_denses, test_groundtruths_denses = get_prediction_classes(denses_model, X_te, y_te)
print(f"Precision: {precision_score(test_groundtruths_denses, test_predictions_denses)}")
print(f"Recall: {recall_score(test_groundtruths_denses, test_predictions_denses)}")
print(f"F1 Score: {f1_score(test_groundtruths_denses, test_predictions_denses)}")

In [ ]:
plot_confusion_matrix(test_predictions_denses, test_groundtruths_denses, class_names)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

def train_and_evaluate_model(model, X_tr, y_tr, X_va, y_va, X_te, y_te, callbacks_list):
    model.fit(X_tr, y_tr,
              batch_size=4096,
              epochs=25,
              validation_data=(X_va, y_va),
              callbacks=callbacks_list)
    model.load_weights('weights.best.keras')

    test_predictions, test_groundtruths = get_prediction_classes(model, X_te, y_te)
    precision = precision_score(test_groundtruths, test_predictions)
    recall = recall_score(test_groundtruths, test_predictions)
    f1 = f1_score(test_groundtruths, test_predictions)
    return precision, recall, f1

def plot_comparison(results):
    models = list(results.keys())
    precisions = [results[model]['precision'] for model in models]
    recalls = [results[model]['recall'] for model in models]
    f1_scores = [results[model]['f1'] for model in models]

    x = range(len(models))

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.bar(x, precisions, color='blue')
    plt.xticks(x, models, rotation='vertical')
    plt.title('Precision Comparison')

    plt.subplot(1, 3, 2)
    plt.bar(x, recalls, color='green')
    plt.xticks(x, models, rotation='vertical')
    plt.title('Recall Comparison')

    plt.subplot(1, 3, 3)
    plt.bar(x, f1_scores, color='red')
    plt.xticks(x, models, rotation='vertical')
    plt.title('F1 Score Comparison')

    plt.tight_layout()
    plt.show()

# Lưu các kết quả vào dictionary
results = {}

# Mô hình với GRU
model_with_gru = model_with_gru(max_len, max_features, embed_size, embedding_matrix)
model_with_gru.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
precision, recall, f1 = train_and_evaluate_model(model_with_gru, X_tr, y_tr, X_va, y_va, X_te, y_te, callbacks_list)
results['GRU'] = {'precision': precision, 'recall': recall, 'f1': f1}

# Mô hình với 2 lớp GRU
model_with_more_gru = model_with_more(max_len, max_features, embed_size, embedding_matrix)
model_with_more_gru.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
precision, recall, f1 = train_and_evaluate_model(model_with_more_gru, X_tr, y_tr, X_va, y_va, X_te, y_te, callbacks_list)
results['Two GRU'] = {'precision': precision, 'recall': recall, 'f1': f1}

# Mô hình với các lớp Dense khác nhau
model_with_denses = model_with_denses(max_len, max_features, embed_size, embedding_matrix)
model_with_denses.compile(optimizer=Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
precision, recall, f1 = train_and_evaluate_model(model_with_denses, X_tr, y_tr, X_va, y_va, X_te, y_te, callbacks_list)
results['Different Dense'] = {'precision': precision, 'recall': recall, 'f1': f1}

# Vẽ biểu đồ so sánh
plot_comparison(results)